# Validation

In order to validate our results, this notebook compare our results with the flood mapping from the ['Global Flood Monitoring'(GFM)](https://services.eodc.eu/browser/#/v1/collections/GFM) for the same area and time range.

In [ ]:
# define time range and area
time_range = "2023-10-11/2023-10-25"
bounding_box = [12.3, 54.3, 13.1, 54.6]

## GFM

In [ ]:
import hvplot.xarray
import pystac_client
from odc import stac as odc_stac
from dask.distributed import Client, wait

# Connect to STAC catalog
eodc_catalog = pystac_client.Client.open("https://stac.eodc.eu/api/v1/")

# Search for available images

search = eodc_catalog.search(collections="GFM", bbox=bounding_box, datetime=time_range)
items_GFM = search.item_collection()

print(f"Found {len(items_GFM)} items") 

### Load the data

In [ ]:
import pyproj

crs = "EPSG:4326"  # Coordinate Reference System - World Geodetic System 1984 (WGS84) 
resolution = 0.00018  # 20 meter in degree

GFM_dc= odc_stac.load(
    items_GFM, 
    bbox=bounding_box,   
    crs=crs,  
    bands=["tuw_likelihood","tuw_flood_extent"],   
    resolution=resolution,   
    dtype='uint8',   
    chunks={"x": 1000, "y": 1000, "time": -1},  
)



### Pre-processing 

In [ ]:
# convert values 255 to NANs
GFM_dc["tuw_flood_extent"] = GFM_dc.tuw_flood_extent.where(GFM_dc.tuw_flood_extent!=255).compute()

# select the band 'tuw_flood_extent'
GFM_dc_flood = GFM_dc["tuw_flood_extent"]

## dask-flood-mapper

In [ ]:
from dask_flood_mapper import flood
from dask.distributed import Client
from bokeh.models import FixedTicker
import hvplot.xarray
from importlib.resources import files
import xarray as xr

#calculate flood decision
fd = flood.decision(bbox=bounding_box, datetime=time_range).compute()



data_text = files("dask_flood_mapper.data").joinpath("wcover.tif")
wcover = xr.open_dataarray(data_text)





### Preprocessing

Make both plots compatible to be plot together.

In [ ]:
fd = fd.where(wcover != 80)
fd_renamed = fd.rename({'x':'longitude', 'y':'latitude'})
fd_renamed_drop_band =fd_renamed.drop_vars('band')

## Validation plot

In [ ]:
import numpy as np
import holoviews as hv
import hvplot.xarray

hv.extension('bokeh')


common_times = np.intersect1d(GFM_dc_flood.time.values, fd.time.values)


def synced_plot(t):
    
    plot1 = GFM_dc_flood.sel(time=t).hvplot.image(x='longitude', y='latitude',title='GFM flood map', cmap=["rgba(0, 0, 1, 0.1)", "darkblue"])
    plot2 = fd_renamed_drop_band.sel(time=t).hvplot.image(x='longitude', y='latitude',title='dask-flood-mapper', cmap=["rgba(0, 0, 1, 0.1)", "darkblue"])

   
    return (plot1 + plot2).cols(2)


combined_plot = hv.DynamicMap(synced_plot, kdims='time').redim.values(time=common_times)

combined_plot